In [14]:
import os, sys
import vcf
import pandas as pd

In [15]:
feature_data = list(pd.read_csv('./demo/test.csv').values)

In [16]:
# search gene name
search_dict = {(266,21555): ['ORF1ab',], 
                              (21563, 25384): ['S',], 
                              (25393, 26220): ['ORF3a',], 
                              (26245, 26472): ['E',], 
                              (26523, 27191): ['M',], 
                              (27202, 27387): ['ORF6',],
                              (27394, 27759): ['ORF7a',], 
                              (27756, 27887): ['ORF7b',], 
                              (27894, 28259): ['ORF8',],
                              (28274, 29533): ['N',], 
                              (29558, 29674): ['ORF10',]}

In [17]:
f = open('./demo/sequence.txt', 'r')
gene_dict = [i.replace('\n','').split('[gbkey=Gene]')[1] for i in f.read().split('>') if len(i)>0 ]
for idx, i in enumerate(search_dict):
    search_dict[i].append(gene_dict[idx])
f.close()

In [18]:
start = 266
end=21555
count = []
while True:
    if start < 21556:
        count.append(start)
        start+=1
    else:
        break

In [19]:
count[-1],count[0], len(count)

(21555, 266, 21290)

In [20]:
for c, next  in zip(count, search_dict[(266,21555)][1]):
    if c>13465 and c < 13475:
        print(c, next)

13466 A
13467 A
13468 C
13469 G
13470 G
13471 G
13472 T
13473 T
13474 T


In [21]:
split_strings = ' '.join(search_dict[(266,21555)][1]).split()
print(split_strings[13200:13205], count[13200:13205])
split_strings.insert(13202, 'C')
print(split_strings[13200:13205], count[13200:13205])

['A', 'A', 'C', 'G', 'G'] [13466, 13467, 13468, 13469, 13470]
['A', 'A', 'C', 'C', 'G'] [13466, 13467, 13468, 13469, 13470]


In [22]:
from Bio.Seq import Seq
class convert_process:
    def __init__(self,):
        # parameter list
        self.chrom = [] #sequence name
        self.position = [] #gene position
        self.ref = [] #before gene
        self.alt = [] #after gene
        self.dp = [] #-
        self.ad = [] #-
        self.gene = [] #RNA name
        self.gene_loc = []
        self.trans = [] #Gene
    def convert_gene(self, or_seq, loc_p, loc_gene):
        if 21555 >= loc_gene[0] and 13468<=loc_gene[0]:
            print(loc_gene)
            split_strings = ''.join(or_seq).split()
            split_strings.insert(13202, 'C')
            or_seq = ''.join(split_strings)
        if loc_p%3==0:
            b_rna = f'{or_seq[loc_p] + or_seq[loc_p+1] + or_seq[loc_p+2]}'.replace(' ','')
            a_rna = f'{loc_gene[1] + or_seq[loc_p+1] + or_seq[loc_p+2]}'.replace(' ','')
        elif loc_p%3==1:
            b_rna = f'{or_seq[loc_p-1] + or_seq[loc_p] + or_seq[loc_p+1]}'.replace(' ','')
            a_rna = f'{or_seq[loc_p-1] + loc_gene[1] + or_seq[loc_p+1]}'.replace(' ','')
        else:
            b_rna = f'{or_seq[loc_p-2] + or_seq[loc_p-1] + or_seq[loc_p]}'.replace(' ','')
            a_rna = f'{or_seq[loc_p-2] + or_seq[loc_p-1] + loc_gene[1]}'.replace(' ','')
        print(b_rna, a_rna)
        try:
            return f'{Seq(b_rna).translate()[0]}2{ Seq(a_rna).translate()[0]}'
        except:
            return f'{Seq(b_rna).translate()[0]}2-'
        
    def search_position(self, pos):
        for idx, i in enumerate(search_dict):
            if (pos[0] >=i[0] and pos[0]<=i[1]) and (pos[1]!=search_dict[i][1][pos[0] - i[0]]):
                self.chrom.append('NC_045512.2')
                self.position.append(pos[0])
                self.ref.append(search_dict[i][1][pos[0] - i[0]])
                self.alt.append(pos[1])
                self.gene.append(search_dict[i][0])
                self.gene_loc.append(pos[0] - i[0]+1)
                self.trans.append(self.convert_gene(search_dict[i][1], pos[0] - i[0], pos))
    def standard(self,):
        # data_vcf = {'CHROM': self.chrom, 'POS': self.position, 'REF': self.ref, 'ALT': self.alt1, 'Gene': self.gene, 'REF_Position': self.debug}
        data_vcf = {'CHROM': self.chrom, 'POS': self.position, 'REF': self.ref, 'ALT': self.alt, 'Gene': self.gene, 'Gene LOC': self.gene_loc, 'Translate': self.trans}
        return pd.DataFrame(data_vcf)
        
start_convert = convert_process()
for i in feature_data:
    start_convert.search_position(i)
    # break
start_convert.standard()

ATG GTG
ATG CTG
ATG GTG
ATG CTG
[13468 ' G']
AAC AAG


,CHROM,POS,REF,ALT,Gene,Gene LOC,Translate
0,NC_045512.2,266,A,G,ORF1ab,1,M2V
1,NC_045512.2,21563,A,C,S,1,M2L
2,NC_045512.2,25393,A,G,ORF3a,1,M2V
3,NC_045512.2,26245,A,C,E,1,M2L
4,NC_045512.2,13468,C,G,ORF1ab,13203,N2K


In [23]:
# LOC from lcl | NC_045512.2_gene_2

# LOC [location=00266..21555] Gene = ORF1ab
# LOC [location=21563..25384] Gene = S
# LOC [location=25393..26220] Gene = ORF3a
# LOC [location=26245..26472] Gene = E
# LOC [location=26523..27191] Gene = M
# LOC [location=27202..27387] Gene = ORF6
# LOC [location=27394..27759] Gene = ORF7a
# LOC [location=27756..27887] Gene = ORF7b
# LOC [location=27894..28259] Gene = ORF8
# LOC [location=28274..29533]  Gene = N
# LOC [location=29558..29674]  Gene = ORF10